In [108]:
import copy as cp

In [1]:
class Game:    
    def __init__(self):
        print("[+] Game Created, please load a game board to proceed...")
        self.board = None
        
    def to_move(self, s):
        self.test_method_failure()
        p = 1 #player
        return p
    
    def terminal_test(self, s):
        self.test_method_failure()
        return True
    
    def utility(self, s, p):
        self.test_method_failure()
        payoff = 1
        return payoff
    
    def actions(self, s):
        '''
            This is only returning empty spaces, available actions should not allow for KO rule breakers
            and maybe even stupid moves
        '''
        self.test_method_failure()
        return [(self.to_move(s), i, j) for i in range(self.n) for j in range(self.n) if go.board[i][j] == 0] 
    
    def result(self, s, a):
        self.test_method_failure()
        state = s
        return state
    
    def load_board(self, s):
        '''
            Loads the board from an opened file stream and saves it to the class field 'board'
            as weel as board dimension 'n', and first player 'p'
            this is done while checking for file integrity
        '''
        
        print("\t[+] Loading Board...")
        contents = s.read().split()
        try:
            self.n = int(contents[0]) # get board dimension
            self.p = int(contents[1]) # get first player to move
            self.board = [list(map(int, lst)) for lst in contents[2:] # load board into 2D list
                          if len(lst)==self.n or                      # check if each row has the correct number of elements
                          any(c in lst for c in ['3', '4', '5', '6', '7', '8', '9'])] #check if any digits above 2 in each row
        except ValueError:
            # if at any point a non numeric value is found failure is issued
            self.load_failure()
            return
        
        if len(self.board) == self.n:
            print("\t[+] Game Board Successfuly Loaded!")
        else:
            # if dimensions dont match return failure
            self.load_failure()
    
    def load_failure(self):
        print("\t\t[-] Bad File...")
        print("\t\t[-] Please try another file...")
        self.board = None
    
    def test_method_failure(self):
        if not self.board:
            print('\t[-] Game not initiated, please lad a game borad...')
            raise Exception('Error: Game not initiated.')

In [363]:
class Cluster:
    
    def __init__(self):
        self.clusters = {1: {}, 2: {}}
        self.board = None
    
    def cluster_maker(self, p, matrix, n, row, column, key):
        if(row<0 or column<0 or row>=n or column>=n):
            return 0
        
        if(matrix[row][column] == 0):
            self.clusters[p][key]["liberties"].update([(row, column)])
            return 0
        
        if(matrix[row][column] != p):
            return 0
        
        matrix[row][column] = -1
        self.clusters[p][key]["pieces"].update([(row, column)])
        
        size = 1
        size += self.cluster_maker(p, matrix, n, row-1, column, key);
        size += self.cluster_maker(p, matrix, n, row+1, column, key);
        size += self.cluster_maker(p, matrix, n, row, column-1, key);
        size += self.cluster_maker(p, matrix, n, row, column+1, key);
        return size
            
        
    def load_board(self, matrix, n):
        maxRegion1 = 0
        maxRegion2 = 0
        self.board = cp.deepcopy(matrix)
        self.n = n
        
        for i in range(n):
            for j in range(n):
                if(matrix[i][j] == 1):
                    key = len(self.clusters[1])+1
                    self.clusters[1].update({key: {"pieces": set(), "liberties": set()}})
                    size1 = self.cluster_maker(1, matrix, n, i, j, key)
                    maxRegion = max(size1, maxRegion1)
                elif(matrix[i][j] == 2):
                    key = len(self.clusters[2])+1
                    self.clusters[2].update({key: {"pieces": set(), "liberties": set()}})
                    size2 = self.cluster_maker(2, matrix, n, i, j, key)
                    maxRegion2 = max(size2, maxRegion2)
                
        return maxRegion1, maxRegion2
    
    def get_liberties(self, row, column):
        liberties = []
        
        if(row-1>=0 and self.board[row-1][column] == 0):
                liberties.append((row-1,column))
                
        if(row+1<self.n and self.board[row+1][column] == 0):
                liberties.append((row+1,column))
        
        if(column-1<0 and self.board[row][column-1] == 0):
                liberties.append((row,column-1))
            
        if(column+1<self.n and self.board[row][column+1] == 0):
                liberties.append((row,column+1))
        return liberties
        
    def join_clusters(self, cluster1, cluster2):
        return {"pieces" : cluster1["pieces"].union(cluster2["pieces"]), "liberties" : cluster1["liberties"].union(cluster2["liberties"])}
    
    def add_2_cluster(self, p, key, row, column):
        liberties = self.get_liberties(row, column) 
        
        try:
            self.clusters[p][key]["liberties"].remove((row,column))
        except:
            pass
                
        self.clusters[p][key]["pieces"].update([(row,column)])
        self.clusters[p][key]["liberties"].update(liberties)
        
    def add_piece(self, p, row, column):
        join_clstrs = []
        
        if(p==1):
            op = 2 
        else: 
            op = 1           
             
        for cluster in self.clusters[p].items():
            if((row,column) in cluster[1]["liberties"]):
                join_clstrs.append(cluster[0])
        print(join_clstrs)
                
        if(len(join_clstrs) > 1):
            for i in join_clstrs[1:]:
                self.clusters[p][join_clstrs[0]] = self.join_clusters(self.clusters[p][join_clstrs[0]], self.clusters[p][i])
                del self.clusters[p][i]
                
            self.add_2_cluster(p, join_clstrs[0], row, column)
            
        elif(len(join_clstrs) == 1):
            self.add_2_cluster(p, join_clstrs[0], row, column)
            
        else:
            self.clusters[p].update({len(self.clusters[p])+1: {"pieces": set([(row,column)]), "liberties": set(self.get_liberties(row, column))}})
            
        for cluster in self.clusters[op].items():
            try:
                cluster[1]["liberties"].remove((row,column)) 
            except:
                pass
            
        self.board[row][column] = p
        
        
                
                
            

In [364]:
go = Game()
go.load_board(open('../boards/board_2.txt', 'r'))
matrix = go.board.copy()
n = go.n
matrix

[+] Game Created, please load a game board to proceed...
	[+] Loading Board...
	[+] Game Board Successfuly Loaded!


[[1, 0, 1, 1], [2, 2, 2, 1], [0, 2, 0, 0], [0, 0, 0, 0]]

In [365]:
clstr = Cluster()
clstr.load_board(matrix, n)
row = 0
column = 1

In [366]:
clstr.clusters[1]

{1: {'pieces': {(0, 0)}, 'liberties': {(0, 1)}},
 2: {'pieces': {(0, 2), (0, 3), (1, 3)}, 'liberties': {(0, 1), (2, 3)}}}

In [367]:
for cluster in clstr.clusters[1].items():
    if(not cluster[1]["liberties"]):
        print("2 won")
    else:
        print("not lost yet")

not lost yet
not lost yet


In [368]:
clstr.clusters[2]

{1: {'pieces': {(1, 0), (1, 1), (1, 2), (2, 1)},
  'liberties': {(0, 1), (2, 0), (2, 2), (3, 1)}}}

In [369]:
clstr.add_piece(1, 3, 0)

[]


In [370]:
clstr.clusters[1]

{1: {'pieces': {(0, 0)}, 'liberties': {(0, 1)}},
 2: {'pieces': {(0, 2), (0, 3), (1, 3)}, 'liberties': {(0, 1), (2, 3)}},
 3: {'pieces': {(3, 0)}, 'liberties': {(2, 0), (3, -1), (3, 1)}}}

In [371]:
clstr.clusters[2]

{1: {'pieces': {(1, 0), (1, 1), (1, 2), (2, 1)},
  'liberties': {(0, 1), (2, 0), (2, 2), (3, 1)}}}

In [372]:
import random 

In [388]:
n5 = [[random.randint(0,2) for j in range(5)] for i in range(5)]

In [389]:
clstr = Cluster()
clstr.load_board(n5, 5)
clstr.clusters[1].keys()

dict_keys([1, 2])

In [390]:
clstr.clusters[2].keys()

dict_keys([1, 2, 3])

In [391]:
for i in range(5):
    for j in range(5):
        if clstr.board[i][j] == 0:
            print(str((i,j)))
            clstr.add_piece(2, i, j)

(0, 0)
[1]
(0, 2)
[1]
(0, 3)
[1, 2]
(1, 0)
[1]
(1, 3)
[1]
(2, 2)
[1]
(2, 3)
[1]
(3, 0)
[1, 3]
(3, 1)
[1]
(3, 2)
[1]
(3, 3)
[1]
(4, 3)
[1]


In [362]:
clstr.clusters[2]

{1: {'pieces': {(0, 2),
   (0, 3),
   (0, 4),
   (1, 1),
   (1, 2),
   (1, 3),
   (1, 4),
   (2, 1),
   (2, 2),
   (2, 3),
   (2, 4),
   (3, 0),
   (3, 1),
   (3, 2),
   (3, 3),
   (3, 4),
   (4, 0),
   (4, 1),
   (4, 2)},
  'liberties': {(4, 3)}},
 3: {'pieces': {(4, 4)}, 'liberties': {(4, 3)}}}

In [259]:
clstr.board

[[1, 2, 2, 1, 2],
 [2, 2, 2, 1, 1],
 [2, 0, 2, 1, 2],
 [1, 2, 0, 1, 2],
 [0, 0, 1, 0, 2]]

In [189]:
len(clstr.clusters[2])

3

In [148]:
clstr.add_piece(1, 1, 2)

In [149]:
n100 = [[random.randint(0,2) for j in range(100)] for i in range(100)]

In [150]:
clstr = Cluster()
clstr.load_board(n100, 100)

(0, 30)

In [151]:
for i in range(100):
    for j in range(100):
        if clstr.board[i][j] == 0:
            clstr.add_piece(1, i, j)

In [152]:
clstr.board

[[1,
  2,
  2,
  2,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  2,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  2,
  2,
  1,
  2,
  1,
  1,
  2,
  2,
  2,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  2,
  2,
  1,
  1,
  1,
  1,
  2,
  2,
  1,
  2,
  2,
  2,
  2,
  1,
  2,
  1,
  1,
  2,
  1,
  1,
  2,
  2,
  1,
  2,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  2,
  1,
  1,
  2,
  1,
  1,
  1,
  2,
  1,
  1,
  2,
  1,
  1,
  2,
  2,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  2,
  1,
  2,
  2,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  1,
  1,
  1,
  2,
  1,
  1,
  2,
  1,
  1,
  2,
  1]